In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [30]:
df = pd.read_csv('imdb_master.csv', encoding = "ISO-8859-1")
del df['Unnamed: 0']
del df['file']
del df['type']

In [31]:
df = df.loc[0:49999]
df = df.sample(frac=1).reset_index(drop=True)

In [32]:
def clean_reviews(no_reviews,dataset):
    import re
    import nltk
    nltk.download('stopwords') 
    from nltk.corpus import stopwords 
    
    my_stopwords = stopwords.words('english') 
            
    from nltk.stem.porter import PorterStemmer

     
    corpus =[] 
    for i in range (0, no_reviews):
        review = re.sub('[^a-zA-Z]',' ', dataset['review'][i]) #Removes all other characters besides the letters and puts the review in a string
        review = review.lower() 
        review = review.split()
        ps = PorterStemmer() 
        
        review = [ps.stem(word) for word in review if not word in set (my_stopwords)]
        review = ' '.join(review) 
        corpus.append(review)
    return corpus

In [33]:
x = clean_reviews(50000,df)

[nltk_data] Downloading package stopwords to /home/aayush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
def bag_of_words(corpus, maximum_features):
    from sklearn.feature_extraction.text import CountVectorizer
    vect = CountVectorizer(ngram_range=(1,3), binary=True)
    X = vect.fit_transform(corpus)
    return X

In [35]:
x

['beyond cloud top artsi group four vignett offer glimps man woman relationship tenuou turbul although film offer superb cinematographi exquisit visual beauti cast fine perform littl meat bone fragment work tast relationship cannot impart full synerg suggest much accomplish one stori hour four nonetheless beyond cloud fodder dilettant visual feast albeit superfici stilt lack substanc',
 'enjoy beauti sceneri movi first time saw dunderklumpen kind cute kiddi corni way remind hrpuffinstuff sat morn swedish backdrop make easi eye expect older kid interest live action anim way behind time older kid get bore definit age set movi nice bit memori us littl kid',
 'movi requir view librarian would librarian best line directli relat librarianship public librari vs academ librari argument classic argument wage among librarian librari school student also break mani librarian stereotyp librarian might even capabl fun even usual sex romanc languag section best movi librarian desk set katherin hepbur

In [36]:
x_data = bag_of_words(x,1000)

In [37]:
x_data

<50000x7869861 sparse matrix of type '<class 'numpy.int64'>'
	with 16493611 stored elements in Compressed Sparse Row format>

In [38]:
labels = df['label']
labels = labels.map({'pos': 1, 'neg': 0})

In [39]:
train_data = x_data[0:34999,:]
test_data = x_data[35000:49999,:]
train_labels = labels[0:34999]
test_labels = labels[35000:49999]

In [40]:
from sklearn.svm import LinearSVC
svm = LinearSVC()

In [41]:
svm.fit(train_data, train_labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [42]:
y_pred_class = svm.predict(test_data)

In [43]:
from sklearn import metrics
metrics.accuracy_score(test_labels, y_pred_class)

0.8957263817587839

In [44]:
metrics.confusion_matrix(test_labels, y_pred_class)

array([[6687,  808],
       [ 756, 6748]])